<a href="https://colab.research.google.com/github/miladmirzazadeh/Gamein-Prototype/blob/master/prototype_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [44]:
import numpy as np 
import pandas as pd 
import random


In [45]:
class Parameters():
  ###################### reading all dataframes from google drive ...... to call them faster and easier in runtime 

  land_stockhouse_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="land_stockhouse", header=0,index_col=0)
  land_machine_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="land_machine", header=0,index_col=0)
  machines_info_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="machines_info", header=0,index_col=0)
  products_volume_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="products_info", header=0,index_col=0)
  bom_df = pd.read_excel("/content/drive/My Drive/projects/gamein_prototype/parameters.xlsx",sheet_name="bom", header=0,index_col=0)

In [46]:
class Player():
  players_name=[]

  def __init__(self, player_name):
    self.name = player_name
    #remove from available
    self.cash = 1000000 # بودجه اولیه
    self.land_objects=dict()

  def calculate_final_net_worth(self):
    #to calculate final net worth
    # inventory_value_sum=sum([self.land_objects[land].calculate_inventory_value() for land in list(self.land_objects.keys())]) #nima jaygozine 3 khate bala
    # net_worth = inventory_value_sum + self.cash 
    net_worth = self.cash #incomplete
    return net_worth 

  def add_land(self, land_id):
    self.land_objects.update({land_id : Land(land_id)}) # یه اینستنس از لند میسازه که ورودی بهش آیدی میدیم . بعد توی خود لند ، بر حسب آیدی بقیه فیچر های لند رو ست میکنیم
    

  def add_machine(self, land_id, machine_id):
    self.land_objects[land_id].add_machine(machine_id)

In [47]:
class Gamein_Bazaar():
  def put_into_market():
    pass

In [48]:
class Land():
  lands_list = [30001,30002,30003,30004,30005,30006,30007,30008,
                30009,30010,30011,30012,30013,30014,30015] #amirreza یه دیتا فریم که یه آی دی زمین داره و ماشین
  available_lands= lands_list #آی دی های باقی مانده
  
  def __init__(self, land_id):
    self.machine = _
    self.id = land_id
    self.cap = Parameters.land_stockhouse_df.loc[self.id, "cap"]
    self.stockhouse = StockHouse(self.cap) # اینجا دیگه دیکشنری نیست چون فرض بر اینه که یه دونه انبار بیشتر نداریم 


    # self.stockhouse = StockHouse()
  def add_machine(self, machine_id):
    self.machine= Machine(machine_id)

In [49]:
class StockHouse():
  def __init__(self, cap):
    self.total_cap = cap
    self.remaining_cap = self.total_cap
    self.inventory_df = pd.DataFrame([], columns= ["total_inventory", "inhand_inventory"]) # indexes = product names 
    # self.cost_per_product = 100

  def add(self, product_name, amount, just_total=False, just_inhand=False): #milad : just_total va just_inhand ezafe shod
    volume = amount * Product.products_volume.loc[product_name, 'volume'] 

    if just_inhand:
      if (product_name in self.inventory_df.index):
        self.inventory_df.loc[product_name,"inhand_inventory"] += amount 
      else:
        self.inventory_df.loc[product_name, "inhand_inventory"] = amount 

    elif just_total:
      if (product_name in self.inventory_df.index):
        self.inventory_df.loc[product_name, "total_inventory"] += amount 
      else:
        self.inventory_df.loc[product_name, "total_inventory"] = amount 
      self.remaining_cap -= volume

    elif not (just_total or just_inhand):# both are False
      if (product_name in self.inventory_df.index):
        self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] += amount 
      else:
        self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] = amount 
      self.remaining_cap -= volume
        
      
      

  def remove(self, product_name, amount, just_total=False, just_inhand=False):
    volume = amount * Product.products_volume.loc[product_name, 'volume'] 
    if not (just_total or just_inhand): # both are False
      self.inventory_df.loc[product_name, ["total_inventory", "inhand_inventory"]] -= amount 
      self.remaining_cap += volumne 

    if (just_total): #total==True, inhand==False
      self.inventory_df.loc[product_name, ["total_inventory"]] -= amount 
      self.remaining_cap += volumne 

    if (just_inhand): 
      self.inventory_df.loc[product_name, ["inhand_inventory"]] -= amount

In [70]:
class Machine():

  def __init__(self,machine_id):
    self.id = machine_id
    self.product_name = Parameters.machines_info_df.loc[self.id, "product"]
    # self.price = _ 
    self.enabled = True # False until machine comes :)
    self.working = 0 # False while mahcine works
    self.remaining_production = 0 # reduce production rate per turn , 
    self.production_cost_per_unit = Parameters.machines_info_df.loc[self.id, "production_cost_per_unit"]
    self.fixed_production_cost = Parameters.machines_info_df.loc[self.id, "fixed_production_cost"]
    self.production_rate = Parameters.machines_info_df.loc[self.id, "production_rate"]
    
  def start_production(self, amount):#milad : ezafe shod
    self.working = 1 
    self.remaining_production = amount
    


In [60]:
#nima mitune nabashe
class Product():

  products_bom = Parameters.bom_df
  products_volume = Parameters.products_volume_df # its a df not a series . so use loc[index, "volume"]  



  # def __init__(self):
  #   self.material_needed = 10
  #   self.lower_band = 1
  #   self.upper_band = 20

In [61]:
class Material():
  Prices = _ #dict (id , price)

In [62]:
class Market(): #nima alan avale kar 

  vitrin = pd.DataFrame([], columns=["seller_obj","product_name", "amount", "price"])

  def reset_vitrin():
    Market.vitrin = pd.DataFrame([], columns=["seller_obj","product_name", "amount", "price"])

In [65]:
class Controller(): 

  # We have to set all initial values here ...... why ? because we wanna reset them everytime we build Controller class 
  Market.reset_vitrin() 
  Land.available_lands = Land.lands_list

  return_message=""
  turn = 0 
  last_turn = 50
  player_objects = dict() #name,object

  def start():  
    Market.reset_vitrin() 
    Land.available_lands = Land.lands_list

    Controller.return_message=""
    Controller.turn = 0 
    Controller.last_turn = 50
    Controller.player_objects = dict() #name,object

  def next_turn():
    Controller.turn+=1 
    #milad : ezafe shod
    for player in list(Controller.player_objects.keys()): #continue production
      machine = list(player.land_objects.values())[0].machine
      stockhouse = list(player.land_objects.values())[0].stockhouse
      if machine.working == 1 : 
        if machine.remaining_production <= machine.production_rate:
          machine.remaining_production = 0
          stockhouse.add(amount=machine.remaining_production, just_inhand=True)
          machine.working = 0 #end production
          print(player +"عملیات تولید تمام شد")
          #CommandAnalyzer.show_message(player_name, "عملیات تولید تمام شد")
        else :
          machine.remaining_production -= machine.production_rate
          stockhouse.add(amount=machine.production_rate, just_inhand=True)

    if Controller.turn == Controller.last_turn : 
      score_board = Controller.score_board()
      Controller.return_message = "End Game. Thank u " + score_board

  
  def score_board():
    net_worths = pd.Series([])
    for player in list(Controller.player_objects.keys()): 
      obj = Controller.player_objects[player] 
      obj.calculate_final_net_worth()
      net_worths[player] = obj 
    net_worths = net_worths.sort(ascending=False)
    return net_worths    

  def create_player(player_name):

    #milad : user's telegram id IS Not his player_name anymore
    #milad : every player can choose a name ..... in CommandAnalyzer we need a dict of telegram username and player name 
    
    if player_name in list(Controller.player_objects.keys()): 
      return ("این نام کاربری پیش از این ثبت شده")
    if len(Land.available_lands) == 0 :
      return ("متاسفانه ظرفیت بازی پر شده ") 
    land_id = random.choice(Land.available_lands) #random از بین لیست زمین های در دسترس
    Land.available_lands.remove(land_id)   #remove from available lands
    machine_id = Parameters.land_machine_df.loc[land_id, "machine_id"] # از دیتافریم زمین و ماشین خوانده میشود

    Controller.player_objects.update({player_name : Player(player_name)})
    Controller.buy_land(player_name, land_id)
    Controller.buy_machine(player_name, land_id, machine_id)
    
  def buy_land(player_name, land_id):
    Controller.player_objects[player_name].add_land(land_id)
    # Controller.player_objects[name].cash -=         اینجا باید پول زمین رو از حسابش کم کنیم . فعلا لازم نیست  چون خودمون بهش زمین رو اختصاص میدیم 

  def buy_machine(player_name, land_id, machine_id):
    Controller.player_objects[player_name].add_machine(land_id, machine_id)

  def show_status(player_name):#milad : in ro ezafe kardam 
    player_obj = Controller.player_objects[player_name]
    stockhouse = list(player_obj.land_objects.values())[0].stockhouse
    machine = list(player_obj.land_objects.values())[0].machine
    return (player_obj.cash, stockhouse.inventory_df, stockhouse.remaining_cap, machine.remaining_production) # we should consider this type of return in CommandAnalyzer()

  def start_production(player_name, product_name, amount):# This Func should take machine_id too (Later Inshallah)
    player_obj = Controller.player_objects[player_name]
    stockhouse = list(player_obj.land_objects.values())[0].stockhouse
    machine = list(player_obj.land_objects.values())[0].machine
    materials = Product.products_bom.loc[product_name]

    # cost = machine.fixed_production_cost + machine.production_cost_per_unit * amount
    cost=0
    volume = Product.products_volume.loc[product_name, "volume"] * amount 

    if not (machine.enabled==True and machine.working==0): ### checking if the required machine is available (free/bought)
      return ('ماشین قادر به تولید نیست')
    if product_name != machine.product_name: ### checking existence
      return (" شما ماشین " + product_name + " را ندارید " ) 
    if player_obj.cash < cost :  ############## Checking Player Cash
      return ("پولت کمه ")
    if stockhouse.remaining_cap < volume: ############### Checking Inventory Remaining Cap
      return ("انبارت اینقدر جا نداره  ")
  
    
    for mat, number in materials.iteritems():
      if number > 0 :
        if (not (mat in stockhouse.inventory_df.index) or (number*amount > stockhouse.inventory_df.loc[mat, "inhand_inventory"])):
          return ("  کافی نیست  " + mat)
        stockhouse.remove(mat, number*amount)
    ##################################### when u get here , everything is OK
    player_obj.cash -= cost
    print("تولید با موفقیت آغاز شد") 
    stockhouse.add(product_name, amount, just_total=True)#milad : bayad mogheye shoroo e tolid be total_inventory ezafe she ama be inhand_inventory na..... yani az remaining cap kam she ke ta vaghti tolid mikone natoone anbaro por kone 
    machine.start_production(amount) #milad : ezafe shod 
    
    
    
  

  def sell_to_gamein(player_name, product_name, amount):#milad : foroosh be gamein ham vase mahsool miani hast ham nahaei . price nadare. 
    pass 

  def put_into_market(player_name, product_name, amount, price):# it doesn't get stockhouse_id or etc. until we have more than one stockhouse
    player_obj = Controller.player_objects[player_name]
    product_on_vitrin_id = random.choice([i for i in range(100) if not(i in Market.vitrin.index)])   # a random unique id , that will assign to vitrin objects  #nima nemifahmam
    stockhouse_obj = list(player_obj.land_objects.values())[0].stockhouse
    

    #check the inventory
    if not (product_name in stockhouse_obj.inventory_df.index):
      return ("Doesn't Exist")

    if stockhouse_obj.inventory_df.loc[product_name, "inhand_inventory"] < amount : 
      return ("Not Enough Inventory")
    
    existing_row = Market.vitrin[np.logical_and(np.logical_and(Market.vitrin.product_name == product_name, Market.vitrin.price == price), Market.vitrin.seller_obj == player_obj)]
    if len(existing_row) != 0:
      # add to existing row IF player_name, product_name and price are equal 
      Market.vitrin.loc[existing_row.index, "amount"] += amount
    else : 
      Market.vitrin.loc[product_on_vitrin_id] = [player_obj, product_name, amount, price]
    stockhouse_obj.remove(product_name, amount, just_inhand=True)
    return ("Done :)")
    
  
  def upgrade_stockhouse(player_name, land_id):
    pass

  def show_product_market_info():
    vitrin = Market.vitrin.copy()
    vitrin["seller_name"] = [obj.name for obj in vitrin.seller_obj]
    vitrin["id"] = vitrin.index
    vitrin = vitrin[["id", "product_name" , "amount", "price", "seller_name"]]
    return vitrin


  def buy_product(player_name, product_on_vitrin_id, amount): # آیدی محصول داخل ویترین رو میده که بریم از روی دیکشنری توی مارکت آبجکتش رو پیدا کنیم و انتقال رو انجام بدیم 
    buyer_player = Controller.player_objects[player_name]
    buyer_stockhouse = list(buyer_player.land_objects.values())[0].stockhouse
    product_on_vitrin = Market.vitrin.loc[product_on_vitrin_id] 
    seller_player = product_on_vitrin.seller_obj 
    volume = Product.products_volume.loc[product_on_vitrin.product_name, 'volume']
    
    if amount*volume > buyer_stockhouse.remaining_cap: 
      return ("فضای انبار کافی نیست")
    if buyer_player.cash < product_on_vitrin.price * amount :
      return ("پول کافی نیست")
    if amount > product_on_vitrin.amount :
      return ("این مقدار کالا در بازار نیست") 
  
    buyer_player.cash -= amount * product_on_vitrin.price #update cash
    buyer_stockhouse.add(product_on_vitrin.product_name, amount) #updata buyer inventory


    #update seller inventory :
    seller_stockhouse = list(seller_player.land_objects.values())[0].stockhouse
    seller_stockhouse.remove(product_on_vitrin.product_name, amount, just_total=True) 
    seller_player.cash += amount * product_on_vitrin.price # add cash to seller player


    #update vitrin
    product_on_vitrin.amount -= amount
    if product_on_vitrin.amount == 0 : #### deleting vitrin row ir amount == 0 
      Market.vitrin.drop(product_on_vitrin_id, inplace=True) 

    return ("معامله انجام شد")

In [66]:
Controller.create_player("milad")

In [69]:
Controller.show_status("milad")[1]

,total_inventory,inhand_inventory
